In [45]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [46]:
csv_file = "Data/ppp_data.csv"
ppp_df = pd.read_csv(csv_file)
ppp_df

,loan_amount,city,zip,naics_code,business_type,lender,cd
0,149985,Richmond,23224,"Administrative, Support, Waste Management & Re...",Partnership,Manufacturers and Traders Trust Company,VA-07
1,149932,HENRICO,23294,"Arts, Entertainment & Recreation",Limited Liability Company(LLC),The Freedom Bank of Virginia,VA-07
2,149914,VIRGINIA BEACH,23457,Construction,Subchapter S Corporation,"Wells Fargo Bank, National Association",VA-02
3,149900,Charlottesville,22903,Other services except Public Administration,Limited Liability Company(LLC),"Blue Ridge Bank, National Association",VA-05
4,149900,HARRISONBURG,22802,Health Care & Social Assistance,Non-Profit Organization,"Summit Community Bank, Inc.",VA-06
...,...,...,...,...,...,...,...
98403,27,MANASSAS PARK,20111,"Administrative, Support, Waste Management & Re...",Limited Liability Company(LLC),"Bank of America, National Association",VA-10
98404,25,ALEXANDRIA,22305,Other services except Public Administration,Limited Liability Company(LLC),"Bank of America, National Association",VA-08
98405,25,ARLINGTON,22204,Construction,Limited Liability Company(LLC),"Wells Fargo Bank, National Association",VA-08
98406,25,CHANTILLY,20152,Retail Trade,Limited Liability Company(LLC),"Bank of America, National Association",VA-10


### Connect to local database

In [15]:
rds_connection_string = "postgres:1234@localhost:5432/ppp_data"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [16]:
engine.table_names()

['ppp_data']

### Virginia - Create new data with select columns

In [23]:
ppp_df.to_sql(name='ppp_df', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the ppp_data table

In [24]:
pd.read_sql_query('select * from ppp_df', con=engine).head()

,loan_amount,city,zip,naics_code,business_type,lender,cd
0,"$149,985",Richmond,23224,"Administrative, Support, Waste Management & Re...",Partnership,Manufacturers and Traders Trust Company,VA-07
1,"$149,932",HENRICO,23294,"Arts, Entertainment & Recreation",Limited Liability Company(LLC),The Freedom Bank of Virginia,VA-07
2,"$149,914",VIRGINIA BEACH,23457,Construction,Subchapter S Corporation,"Wells Fargo Bank, National Association",VA-02
3,"$149,900",Charlottesville,22903,Other services except Public Administration,Limited Liability Company(LLC),"Blue Ridge Bank, National Association",VA-05
4,"$149,900",HARRISONBURG,22802,Health Care & Social Assistance,Non-Profit Organization,"Summit Community Bank, Inc.",VA-06


In [47]:
len(list(ppp_df["cd"].values))

98408

In [48]:
len(set(ppp_df["cd"].values))

19

In [49]:
ppp_va = sorted(list(set(ppp_df["cd"].values))[1:])[6:-1]
ppp_va

['VA-01',
 'VA-02',
 'VA-03',
 'VA-04',
 'VA-05',
 'VA-06',
 'VA-07',
 'VA-08',
 'VA-09',
 'VA-10',
 'VA-11']

In [55]:
cd_data = []

for cd in ppp_va:
    cd_list = list(ppp_df[ppp_df["cd"]==cd]["loan_amount"].values)
    cleaned_cd_list = [loan for loan in cd_list if str(loan) != 'nan']
    cd_total = sum(cleaned_cd_list)
    cd_dict = {
        "y":cd_total,
        "label":cd,
    } 
    cd_data.append(cd_dict)

cd_data

[{'y': 245123079, 'label': 'VA-01'},
 {'y': 265217455, 'label': 'VA-02'},
 {'y': 190516757, 'label': 'VA-03'},
 {'y': 257982591, 'label': 'VA-04'},
 {'y': 296615520, 'label': 'VA-05'},
 {'y': 243769129, 'label': 'VA-06'},
 {'y': 437968361, 'label': 'VA-07'},
 {'y': 321780571, 'label': 'VA-08'},
 {'y': 187991464, 'label': 'VA-09'},
 {'y': 551924420, 'label': 'VA-10'},
 {'y': 359333838, 'label': 'VA-11'}]